# **ONSAGER TESTING**

## **Installs**

### Firedrake

In [14]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2026-02-04 16:13:59--  https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4775 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2026-02-04 16:13:59 (59.9 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4775/4775]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ set +x
























#########################################################

### Netgen

In [7]:
try:
    !wget "https://fem-on-colab.github.io/releases/ngsolve-install-release-real.sh" -O "/tmp/ngsolve-install.sh"
    !bash "/tmp/ngsolve-install.sh"
    import ngsolve  # noqa: F401
except:
    import netgen  # noqa: F401

--2026-02-04 16:06:54--  https://fem-on-colab.github.io/releases/ngsolve-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4214 (4.1K) [application/x-sh]
Saving to: ‘/tmp/ngsolve-install.sh’

/tmp/ngsolve-instal 100%[===================>]   4.12K  --.-KB/s    in 0s      

2026-02-04 16:06:54 (52.9 MB/s) - ‘/tmp/ngsolve-install.sh’ saved [4214/4214]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ NGSOLVE_INSTALLED=/usr/local/share/fem-on-colab/ngsolve.installed
+ [[ ! -f /usr/local/share/fem-on-colab/ngsolve.installed ]]
+ OCC_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/f2f91b79/releases/occ

### Other

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## **Code**

In [11]:
from netgen.geom2d import SplineGeometry
from firedrake import Mesh

In [13]:
from netgen.geom2d import SplineGeometry
from firedrake import Mesh

L = 1.0
r = 0.2
h = 0.05

geo = SplineGeometry()

geo.AddRectangle(
    p1=(-L, -L),
    p2=( L,  L),
    bc="outer"
)

geo.AddCircle(
    c=(0.0, 0.0),
    r=r,
    leftdomain=0,
    rightdomain=1,
    bc="hole"
)

ngmesh = geo.GenerateMesh(maxh=h)

# Write Netgen mesh to file
ngmesh.Save("square_with_hole.vol")

# Read into Firedrake
mesh = Mesh("square_with_hole.vol")


RuntimeError: Mesh file square_with_hole.vol has unknown format 'vol'.

In [ ]:
def navier_stokes(h=2**-10, degree=2, nu=0.01, timestep=2**-5, end_time=1.0):
    mesh = RectangleMesh(50, 50, 2.0, 1.0)
    cylinder = Circle(Point(0.5, 0.5), 0.1)
    mesh = MeshGenerator(mesh, [cylinder])
    n = FacetNormal(mesh)
    x, y = SpatialCoordinate(mesh)

    V = VectorFunctionSpace(mesh, 'CG', degree)
    Q = FunctionSpace(mesh, 'CG', degree - 1)
    W = V * Q

    u = Function(V)
    p = Function(Q)
    u_old = Function(V)

    bc_inlet = DirichletBC(W.sub(0), Constant((1.0, 0.0)), 'near(x[0], 0)')
    bc_walls = DirichletBC(W.sub(0), Constant((0.0, 0.0)), 'on_boundary')
    bcs = [bc_inlet, bc_walls]

    F = (inner((u - u_n) / timestep, v) * dx +
         inner(grad(u) * u, v) * dx +
         inner(grad(v), p) * dx -
         inner(div(u), q) * dx)

    sp = {
        'snes_max_it': 100,
    }

    fig, ax = plt.subplots(figsize=(10, 6))
    energy = []
    state = {'t': 0.0}

    def update(frame):
        if frame > 0:
            state['t'] += timestep
            print(f'Solving for time t = {state["t"]:.4f}:')
            solve(F == 0, u, bcs=bcs, solver_parameters=sp)
            u_n.assign(u)

            # Record energy
            energy.append(0.5 * assemble(inner(u, u) * dx))

        ax.clear()
        plot(u, axes=ax, linewidth=3)
        ax.set_title(f'Navier-Stokes (t = {state["t"]:.2f})')
        ax.set_xlabel('x')
        ax.set_ylabel('u')
        ax.set_ylim(-1, 1.5)
        ax.grid(True)

    num_frames = int(end_time / timestep) + 1
    anim = FuncAnimation(fig, update, frames=num_frames, interval=100)
    plt.close()

    # Plot energy
    plt.figure()
    plt.plot(np.arange(len(energy)) * timestep, energy)
    plt.title('Energy over time')
    plt.xlabel('Time')
    plt.ylabel('Energy')
    plt.grid(True)
    plt.show()

    return HTML(anim.to_jshtml())
